# 変分法と変分量子固有値ソルバー法を学習する

この実習では、変分法の基本的な考え方と、その方法に基づいた変分量子回路と呼ばれる量子計算の手法を学びます。特に、量子計算と古典計算を組み合わせた「**量子・古典ハイブリッドアルゴリズム**」としての変分量子回路に着目します。この手法を用いて、近似的な固有値計算を可能にする**変分量子固有値ソルバー法**と呼ばれる方法へ拡張していきます。

この教材は、Qiskit textbookの["Simulating Molecules using VQE"](https://qiskit.org/textbook/ch-applications/vqe-molecules.html)を参考にしています。

```{contents} 目次
---
local: true
---
```

## はじめに<a id='introduction'></a>
行列で表現されるある物理系に対して、その最も小さい固有値を見つけるという操作は、多くのアプリケーションで必要となる重要な技術です。例えば化学の計算では、分子を特徴づけるエルミート行列の最小固有値はそのシステムの最もエネルギーの低い状態（基底状態）のエネルギーになります。最小固有値を見つけるには「**量子位相推定**」[1]と呼ばれる手法（この[ノートブック](shor.ipynb#qpe)を参照）を使うことができますが、この手法を使って実用的な問題を解こうとすると、そのために必要な量子回路はNISQコンピュータでは実現できないほど長くなることが知られています。そのために、短い量子回路を利用して分子の基底状態エネルギーを推定する手法として、**変分量子固有値ソルバー法**（*Variational Quantum Eigensolver*, VQE）が提案されました [[2]](https://www.nature.com/articles/ncomms5213)。

まず、VQEの元になる関係を形式的に表現してみましょう。何か分からない最小固有値$\lambda_{min}$とその固有状態$|\psi_{min}\rangle$をもったエルミート行列$H$が与えられたとして、VQEはその系のエネルギーの下限である$\lambda_{min}$の近似解$\lambda_{\theta}$を求める手法です。つまり

$$
\begin{aligned}
    \lambda_{min} \le \lambda_{\theta} \equiv \langle \psi(\theta) |H|\psi(\theta) \rangle
\end{aligned}
$$

を満たす、できるだけ小さい$\lambda_{\theta}$を求めることに対応します。ここで$|\psi(\theta)\rangle$は近似解$\lambda_{\theta}$に対応する固有状態で、$\theta$はパラメータです。つまり、適当な初期状態$|\psi\rangle$にユニタリー$U(\theta)$で表現されるパラメータ化された回路を適用することで、$|\psi_{min}\rangle$を近似する状態$|\psi(\theta)\rangle \equiv U(\theta)|\psi\rangle$を得ようというアイデアです。最適なパラメータ$\theta$の値は、期待値 $\langle \psi(\theta) |H|\psi(\theta) \rangle$が最小になるように古典計算を繰り返しながら求めていくことになります。


## 量子力学における変分法<a id='varmethod'></a>
### 背景<a id='backgroundmath'></a>

VQEは量子力学の**変分法**を応用した手法です。変分法をより良く理解するために、基礎的な数学的背景を見てみましょう。

行列$A$の固有ベクトル$|\psi_i\rangle$とその固有値$\lambda_i$は、$A |\psi_i\rangle = \lambda_i |\psi_i\rangle$という関係を持っていますね。行列$H$がエルミート行列$H = H^{\dagger}$の場合、スペクトル定理から$H$の固有値は実数になります（$\lambda_i = \lambda_i^*$）。実際に実験で測定できる量は実数である必要があるため、量子系のハミルトニアンを記述するためにはエルミート行列が適切です。さらに、$H$は以下のように表現することもできます。

$$
H = \sum_{i = 1}^{N} \lambda_i |\psi_i\rangle \langle \psi_i |
$$

ここで、各$\lambda_i$は固有ベクトル$|\psi_i\rangle$に対応する固有値です。任意の量子状態$|\psi\rangle$に対して観測量$H$を測定した時の期待値は、以下の式で与えられます。

$$
\langle H \rangle_{\psi} \equiv \langle \psi | H | \psi \rangle
$$

上式の$H$を期待値の式に代入すると

$$
\begin{aligned}
\langle H \rangle_{\psi} = \langle \psi | H | \psi \rangle &= \langle \psi | \left(\sum_{i = 1}^{N} \lambda_i |\psi_i\rangle \langle \psi_i |\right) |\psi\rangle\\
&= \sum_{i = 1}^{N} \lambda_i \langle \psi | \psi_i\rangle \langle \psi_i | \psi\rangle \\
&= \sum_{i = 1}^{N} \lambda_i | \langle \psi_i | \psi\rangle |^2
\end{aligned}
$$

になります。最後の式は、任意の状態$|\psi\rangle$に対する$H$の期待値は、$\lambda_i$を重みとした固有ベクトル$|\psi_i\rangle$と$|\psi\rangle$の内積（の絶対値二乗）の線形結合として与えられることを示しています。この式から、$| \langle \psi_i | \psi\rangle |^2 \ge 0$ であるために

$$
\lambda_{min} \le \langle H \rangle_{\psi} = \langle \psi | H | \psi \rangle = \sum_{i = 1}^{N} \lambda_i | \langle \psi_i | \psi\rangle |^2
$$

が成り立つことは明らかです。上記の式が**変分法**と呼ばれるもの（テキストによっては**変分原理**と呼ぶ）[3]で、波動関数を”うまく取る”ことで、ハミルトニアン$H$の期待値の下限として最小固有値を近似的に求めることができることを表しています。この式から、$|\psi_{min}\rangle$状態の期待値は$\langle \psi_{min}|H|\psi_{min}\rangle = \langle \psi_{min}|\lambda_{min}|\psi_{min}\rangle = \lambda_{min}$になることも分かるでしょう。


### 基底状態の近似<a id='groundstate'></a>
系のハミルトニアンがエルミート行列$H$で表現されている場合、系の基底状態のエネルギーは$H$の最小固有値になります。まず$|\psi_{min}\rangle$の初期推定として任意の波動関数$|\psi \rangle$（*Ansatz*と呼ばれる）を選び、その状態での期待値$\langle H \rangle_{\psi}$を計算します。変分法の鍵は、この期待値が小さくなるように波動関数を更新しながら計算を繰り返し、ハミルトニアンの基底状態エネルギーに近づけていくところにあります。

## 変分量子固有値ソルバー法<a id='vqe'></a>
### 変分量子回路<a id='varforms'></a>
量子コンピューター上で変分法を実装するには、系統的に*Ansatz*を更新する方法が必要です。VQEはこれを決まった構造を持つパラメータ化された量子回路（**変分量子回路**）を使って行います。この回路はしばしば**変分フォーム**（*variational form*）とも呼ばれ、ユニタリー変換$U(\theta)$で表現されます（$\theta$はパラメータ）。
変分フォームを初期状態$|\psi\rangle$（例えば標準状態$|0\rangle$）に適用すると、出力として$|\psi(\theta)\rangle \equiv U(\theta)|\psi\rangle$が生成されます。この状態の元で期待値$\langle \psi(\theta)|H|\psi(\theta)\rangle$が$\lambda_{min}$に近付くように、$|\psi(\theta)\rangle$に対してパラメータ$\theta$の最適化を行います。
パラメータの最適化は古典計算で実行することを想定しており、その意味でVQEは典型的な**「量子・古典ハイブリッドアルゴリズム」**の一つです。

変分フォームの決め方ですが、解きたい問題のドメインに応じて特定の構造を持つ変分フォームを導入することがあります。そうではなく、幅広い問題への応用ができるようにドメインに依存しない形の変分フォーム（例えば$R_Y$ゲート）を使うこともあります。高エネルギー実験への応用では、この$R_Y$ゲートを使った変分フォームを実装します。

### 単純な変分フォーム<a id='simplevarform'></a>
変分フォームを決める時には、2つの相反する目的に対してバランスを考える必要があります。$n$量子ビットの変分フォームは、パラメータの数を増やせば$|\psi\rangle \in \mathbb{C}^N$（$N=2^n$）の任意の状態ベクトル$|\psi\rangle$を生成できるでしょう。しかし、パラメータの最適化のことを考えれば、できれば可能な限り少ないパラメータで変分フォームを構築したいですよね。

ここでは、まず$n=1$の場合を考えます。$U3$ゲートは3つのパラメータ$\theta$、$\phi$、$\lambda$を使って以下の変換を表現します:

$$
U3(\theta, \phi, \lambda) = \begin{pmatrix}\cos\frac{\theta}{2} & -e^{i\lambda}\sin\frac{\theta}{2} \\ e^{i\phi}\sin\frac{\theta}{2} & e^{i\lambda + i\phi}\cos\frac{\theta}{2} \end{pmatrix}
$$

グローバル位相を除いて、3つのパラメータを適切に設定して実装すれば任意の単一量子ビットの変換が行えます。この**ユニバーサル**な変分フォームは、3つしかパラメータがないため効率的に最適化できるという特徴があります。ただ強調すべき点は、任意の状態を生成できるということは、この変分フォームが生成する状態は$H$の期待値を計算する上で必要になる状態に限定されないということです。つまり、この性質が意味するのは、最小の期待値が求まるかどうかは古典計算の最適化の能力だけに依存するということです。

### パラメータの最適化<a id='optimization'></a>
パラメータ化された変分フォームを選択したら、ターゲットとなるハミルトニアンの期待値を最小化するように、変分法に従ってパラメータを最適化する必要があります。パラメータの最適化のプロセスには様々な課題があります。例えば、量子ハードウェアには様々なタイプのノイズがあるため、その状態でエネルギーを測定しても正しい答えが返ってくるという保証はありません。そのために、パラメータの最適化に使う目的関数の評価が実際のエネルギーの値からずれてしまい、正しくパラメータの更新ができない可能性があります。また、最適化の手法（**オプティマイザー**）によっては、パラメータの数に依って目的関数を評価する回数が増えることがあり、さらにノイズの影響を受けやすくなります。つまり、アプリケーションの要求を考慮しながら、オプティマイザーの選択にも気を配る必要があります。

最も一般的な最適化手法は、エネルギーの減少が極大になるような方向に各パラメータを更新する**勾配降下法**です。各パラメータごとに勾配を計算するため、最適化すべきパラメータの数に応じて目的関数を評価する回数は増えます。また、この性質から探索空間の中で局所的な最適値を素早く発見することは可能ですが、逆に探索が局所的な最小値に留まってしまうことがあります。勾配降下法は直感的で理解しやすい最適化の手法ですが、少なくとも現在のNISQコンピュータでは精度良く実行するのは難しいと考えられていて、現状ではあまり推奨されてはいません。

ノイズのある量子コンピュータで目的関数を最適化する適切なオプティマイザーとして、*Simultaneous Perturbation Stochastic Approximation*（**SPSA**）があります。SPSAは２回の測定だけで目的関数の勾配を近似できるという特徴があります。勾配降下法では各パラメータを独立に変化させるのに対して、SPSAでは全てのパラメータを同時にランダムに変化させます。以上のことから、現在のところVQEを利用する場合のオプティマイザーとしてはSPSAが推奨されているようです。

ノイズがない量子コンピュータで目的関数を評価する場合（例えば状態ベクトルシミュレータで実行する場合など）は、Pythonの[SciPy](https://www.scipy.org/scipylib/index.html)パッケージで提供されているオプティマイザーなど様々な選択肢があります。この実習では、Qiskit Aquaでサポートされているオプティマイザーの中で、特に*Constrained Optimization by Linear Approximation*（**COBYLA**）と呼ばれるオプティマイザーも使用します。COBYLAは目的関数の評価を1回しか実行しない（つまり評価の回数がパラメータの数に依存しない）ため、ノイズがない状態でかつ評価の回数を少なくしたい場合にはCOBYLAの利用が推奨されているようです。いづれにしろ、どのオプティマイザーがベストかはVQEアルゴリズムの実装形式や実行環境によって変わるため、ある程度経験によって決める必要があると考えられます。

### 変分フォームを使った実例<a id='example'></a>
ではここで、単一量子ビットの変分フォームを利用してパラメータ最適化の例を実行してみましょう。例として、ランダムな確率分布のベクトル$\vec{x}$を入力として与えた時、出力の確率分布が$\vec{x}$に近くなるように単一量子ビットの変分フォームを決定するという問題を考えます（2つの確率分布の近さはL1距離によって定義します）。

```{image} figs/vqe_u3.png
:alt: vqe_u3
:class: bg-primary mb-1
:width: 400px
:align: center
```

最初に、Pythonでランダムな確率分布のベクトルを作成します。

In [ ]:
# Tested with python 3.7.9, qiskit 0.23.5, numpy 1.20.1

import numpy as np
import matplotlib.pyplot as plt

np.random.seed(999999)
target_distr = np.random.rand(2)
target_distr /= sum(target_distr)

次に、単一の$U3$変分フォームの3つのパラメータを引数として受け取り、対応する量子回路を返す関数を定義します。

In [2]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
def get_var_form(params):
    qr = QuantumRegister(1, name="q")
    cr = ClassicalRegister(1, name='c')
    qc = QuantumCircuit(qr, cr)
    qc.u(params[0], params[1], params[2], qr[0])
    qc.measure(qr, cr[0])
    return qc

変分フォームのパラメータのリストを入力とし、パラメータに対応したコストを計算する目的関数を定義します。アルゴリズムを実行するバックエンドとして、**QASMシミュレータ**を使用します。

In [3]:
from qiskit import Aer, execute
backend = Aer.get_backend("qasm_simulator")
NUM_SHOTS = 10000

def get_probability_distribution(counts):
    output_distr = [v / NUM_SHOTS for v in counts.values()]
    if len(output_distr) == 1:
        output_distr.append(0)
    return output_distr

def objective_function(params):
    qc = get_var_form(params)
    result = execute(qc, backend, shots=NUM_SHOTS).result()
    output_distr = get_probability_distribution(result.get_counts(qc))
    cost = sum([np.abs(output_distr[i] - target_distr[i]) for i in range(2)])
    return cost

最後にCOBYLAオプティマイザーのインスタンスを作成し、アルゴリズムを実行します。出力される確率分布は実行の度に異なり、ターゲットの確率分布と完全には同じにならないことに注意してください。出力の精度は量子計算の回数（ショット数＝NUM_SHOTS）に依存するので、ショット数を増減させた時の一致具合を確認してみてください。

In [4]:
from qiskit.aqua.components.optimizers import COBYLA

optimizer = COBYLA(maxiter=500, tol=0.0001)

params = np.random.rand(3)
ret = optimizer.optimize(num_vars=3, objective_function=objective_function, initial_point=params)

qc = get_var_form(ret[0])
counts = execute(qc, backend, shots=NUM_SHOTS).result().get_counts(qc)
output_distr = get_probability_distribution(counts)

print("Target Distribution:", target_distr)
print("Obtained Distribution:", output_distr)
print("Output Error (L1-Distance):", ret[1])
print("Parameters Found:", ret[0])

Target Distribution: [0.51357006 0.48642994]
Obtained Distribution: [0.5353, 0.4647]
Output Error (Manhattan Distance): 0.020459881261160884
Parameters Found: [ 1.52294877 -0.07797282  0.65499835]


## 参考文献<a id='references'></a>
1. Nielsen, Michael A and Chuang, Isaac L, "Quantum Computation and Quantum Information", Cambridge University Pres, 2000.
2. Peruzzo, Alberto, et al., "A variational eigenvalue solver on a photonic quantum processor", [Nature commun. 5, 4213 (2014)](https://www.nature.com/articles/ncomms5213).
